# Agentic Financial Analysis: Report Generator

This notebook showcases an **agentic financial analysis system** powered by **AutoGen**. Multiple specialized agents collaborate in a GroupChat to research and generate a Python-based report analyzing **Nvidia’s stock price performance** over the past month.

The agents divide responsibilities such as:
- **Task planning**
- **Data retrieval**
- **Code execution**
- **Report writing**

— all while keeping the **user in the loop** to approve or redirect as needed.

A separate script (`fetch_nvidia_news.py`) also enables external news sourcing via the **NewsAPI**.

By structuring the system with **reusable components** and **feedback-aware behavior**, this project demonstrates how **autonomous agents** can handle complex financial analysis workflows with **minimal manual intervention**.

## Setup

In [1]:
import os
import autogen

llm_config = {
    "config_list": [{
        "model": "gpt-4o",
        "api_key": os.environ["OPENAI_API_KEY"], 
    }]
}

In [2]:
from autogen import ConversableAgent, AssistantAgent

## Define Task

In [3]:
task = (
    "Write a blog post analyzing Nvidia’s stock performance over the past month (from 2025-05-23 to 2025-06-23). "
    "Include key metrics such as highest/lowest/closing prices, volatility, and any significant news or events "
    "that may have influenced the stock. Summarize the findings in markdown format, suitable for a financial blog."
)


## Create Agents

The following agents will participate in a chat:

### 1. **User_proxy** or **Admin**: enables optional human-in-the-loop feedback and task finalization

In [4]:
user_proxy = autogen.ConversableAgent(
    name="Admin",
    system_message=(
        "You are the admin responsible for giving the task and reviewing the blog post. "
        "After the blog is drafted, provide feedback and ask the Writer to revise it accordingly."
    ),
    code_execution_config=False,
    llm_config=llm_config,
    human_input_mode="ALWAYS",
)


### 2. **Planner**: to determine relevant information needed to complete the task.

In [5]:
planner = autogen.ConversableAgent(
    name="Planner",
    system_message=(
        "Your role is to break down the given task into specific, actionable steps. "
        "All data must be retrievable via Python code, including using APIs like yfinance for stock data or NewsAPI for news. "
        "Clearly describe what each step should accomplish, and verify progress from other agents. "
        "If a step fails, propose a workaround to keep the task moving forward."
    ),
    description="Planner: Break the task into Python-retrievable steps and guide the process.",
    llm_config=llm_config,
)


### 3. **Engineer**: to write code using the defined plan by the planner.

In [6]:
engineer = autogen.AssistantAgent(
    name="Engineer",
    system_message=(
        "You are responsible for writing Python code that fulfills the Planner's instructions. "
        "Make sure the code is efficient, readable, and includes comments when needed. "
        "If external data is needed (e.g., stock prices, news), use libraries like yfinance or requests to fetch it."
    ),
    description="Engineer: Writes Python code to fulfill Planner instructions.",
    llm_config=llm_config,
)

### 4. **Executor**: to execute the code written by the engineer.

In [7]:
executor = autogen.ConversableAgent(
    name="Executor",
    system_message=(
        "Execute the code provided by the Engineer and return the result. "
        "If execution fails, report the error and any traceback so the team can debug it."
    ),
    description="Executor: Runs Python code and reports outputs or errors.",
    human_input_mode="NEVER",
    code_execution_config={
        "last_n_messages": 3,
        "work_dir": "coding",
        "use_docker": False,
    },
    llm_config=llm_config,
)

### 5. **Writer**: to write the report.

In [8]:
writer = autogen.ConversableAgent(
    name="Writer",
    system_message=(
        "Write a clear and engaging blog post using markdown, based on the data and insights provided. "
        "Use relevant titles, bullet points, and code blocks formatted as ```md```. "
        "After the Admin provides feedback, update the post accordingly."
    ),
    description="Writer: Drafts and revises a blog post based on the executed results.",
    llm_config=llm_config,
)

## Build the GroupChat

- The group chat is constructed using all defined agents
- The `GroupChatManager` handles the orchestration automatically
- We initialize a GroupChat where agents collaborate without any enforced communication rules. Each agent autonomously decides when to speak, enabling a freeform conversation flow.

In [9]:
from autogen import GroupChat, GroupChatManager

In [10]:
groupchat = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
)

#### Add a Manager

In [11]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, llm_config=llm_config
)


## Initiate the Chat

In [12]:
groupchat_result = user_proxy.initiate_chat(
    manager,
    message=task,
)

Admin (to chat_manager):

Write a blog post analyzing Nvidia’s stock performance over the past month (from 2025-05-23 to 2025-06-23). Include key metrics such as highest/lowest/closing prices, volatility, and any significant news or events that may have influenced the stock. Summarize the findings in markdown format, suitable for a financial blog.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To analyze Nvidia's stock performance over the past month and produce a blog post in markdown format, we can break down the task into the following steps:

### Step 1: Fetch Nvidia's Stock Data
1. **Objective**: Retrieve Nvidia's historical stock prices from 2025-05-23 to 2025-06-23.
2. **Method**: Use the `yfinance` library in Python to download the stock data.
   - Code:
     ```python
     import yfinance as yf

     # Fetch historical data for Nvidia (NVDA)
     nvda = yf.Ticke

## Adding a Speaker Selection Policy

To ensure a controlled flow of communication between agents, we implemented a custom speaker selection policy using the `allowed_or_disallowed_speaker_transitions` parameter in AutoGen's `GroupChat`. This policy enforces structured interactions between agents, preventing redundant or chaotic exchanges.

The order of interaction follows this guided sequence:
```
UserProxy → Planner → Engineer → Executor → Writer → UserProxy
```

In [13]:
groupchat_with_policy = autogen.GroupChat(
    agents=[user_proxy, engineer, writer, executor, planner],
    messages=[],
    max_round=10,
    allowed_or_disallowed_speaker_transitions={
        user_proxy: [engineer, writer, executor, planner],
        engineer: [user_proxy, executor],
        writer: [user_proxy, planner],
        executor: [user_proxy, engineer, planner],
        planner: [user_proxy, engineer, writer],
    },
    speaker_transitions_type="allowed",
)

## Initiate Chat

In [14]:
manager2 = autogen.GroupChatManager(
    groupchat=groupchat_with_policy, llm_config=llm_config
)

groupchat_with_policy_result = user_proxy.initiate_chat(
    manager2,
    message=task,
)

Admin (to chat_manager):

Write a blog post analyzing Nvidia’s stock performance over the past month (from 2025-05-23 to 2025-06-23). Include key metrics such as highest/lowest/closing prices, volatility, and any significant news or events that may have influenced the stock. Summarize the findings in markdown format, suitable for a financial blog.

--------------------------------------------------------------------------------

Next speaker: Planner


>>>>>>>> USING AUTO REPLY...
Planner (to chat_manager):

To complete the task of analyzing Nvidia's stock performance over the past month, we need to execute a series of steps, from acquiring the necessary stock data to summarizing key findings in a markdown format for your blog post.

### Step-by-step Breakdown:

#### Step 1: Retrieve Stock Data
**Objective:** Gather Nvidia’s stock data from 2025-05-23 to 2025-06-23, including daily opening, closing, high, and low prices, along with trading volume.

- **Action:** Use the `yfinance` libr

Replying as Admin. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  wow great


Admin (to chat_manager):

wow great

--------------------------------------------------------------------------------

Next speaker: Writer


>>>>>>>> USING AUTO REPLY...
Writer (to chat_manager):

Fantastic! With the successful execution of the code, we now have the necessary data to craft the markdown blog post.

Here’s the comprehensive Nvidia stock performance analysis based on the results:

```markdown
# Nvidia Stock Performance Analysis (2025-05-23 to 2025-06-23)

## Overview
Over the course of the past month, Nvidia's stock has demonstrated notable fluctuations influenced by various economic and sector-specific dynamics. Here's a closer look at the key performance metrics and influencing events.

## Key Metrics
- **Highest Price:** $146.20
- **Lowest Price:** $129.15
- **Closing Price on June 23, 2025:** $143.85
- **Volatility (Standard Deviation of Closing Prices):** 4.07

## Key Events and News
Several significant events and news stories likely impacted Nvidia's stock performa

## Why a Speaker Selection Policy Matters

In this notebook, we tested two different modes of agent collaboration to complete a financial analysis task using AutoGen's `GroupChat`. In the first chat without a policy (`speaker_transitions_type="auto"`), the agents struggled to complete the defined task, resulting in unexecuted code and an incomplete report.

By introducing this explicit speaker selection policy, agents followed a well-defined handoff procedure that aligned with their responsibilities. As a result, the system successfully:

- Fetched Nvidia stock and news data
- Executed calculations
- Generated a polished markdown blog post summarizing insights
  
This demonstrates how **structured agent collaboration** improves reliability, task completion, and output quality in multi-agent systems.